# Semantic Text Similarity

In [139]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# Requisites
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
import numpy as np

In [140]:
from typing import Tuple, List, Optional

In [142]:
WORD_EMBEDDING_FILE = 'C:/Users/ralva/Downloads/cc.es.300.bin.gz'
from gensim.models import fasttext
wv_model = fasttext.load_facebook_vectors(WORD_EMBEDDING_FILE)

In [ ]:
USE_MMAP = False
if USE_MMAP:
    from gensim.models.fasttext import FastTextKeyedVectors
    MMAP_PATH = 'cc.es.gensim.bin'
    wv_model.save(MMAP_PATH)
    wv_model = FastTextKeyedVectors.load(MMAP_PATH, mmap='r')

In [6]:
# Example
input_pairs_example = [
    ('Me gusta el futbol', 'Disfruto viendo partidos de futbol', 4),
    ('El cielo está despejado', 'Hace un día bonito', 4.5),
    ('Me encanta viajar', 'Explorar nuevos lugares es una pasión', 3.5),
    ('Prefiero el verano', 'No me gusta el frío del invierno', 2.5),
    ('Tengo hambre', '¿Qué hay para cenar?', 2),
    ('La música me relaja', 'Escuchar música es una terapia', 3),
    ('El libro es emocionante', 'No puedo dejar de leerlo', 4),
    ('Me gusta la pizza', 'Es mi comida favorita', 4.5),
    ('Estoy cansado', 'Necesito hacer una siesta', 1.5),
    ('Hoy hace mucho calor', 'Es un día sofocante', 3.5)
]

In [143]:
REMAP_EMBEDDINGS: bool = True
USE_PRETRAINED: bool = True
MAX_LEN: int = 96

In [144]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset("PlanTL-GOB-ES/sts-es")

In [145]:
input_pairs = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["train"].to_list()]
input_pairs_val = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["validation"].to_list()]
input_pairs_test = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["test"].to_list()]

In [146]:
all_input_pairs = input_pairs + input_pairs_val + input_pairs_test
# Preprocesamiento de las oraciones y creación del diccionario
sentences_1_preproc = [simple_preprocess(sentence_1) for sentence_1, _, _ in all_input_pairs]
sentences_2_preproc = [simple_preprocess(sentence_2) for _, sentence_2, _ in all_input_pairs]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))
# Versión aplanada para poder entrenar el modelo
sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc
diccionario = Dictionary(sentences_pairs_flattened)

In [147]:
print("Max Len:", max([len(s) for s in sentences_1_preproc]), max([len(s) for s in sentences_2_preproc]))
print(list(diccionario.doc2idx(sentences_1_preproc[0])))

Max Len: 87 86
[11, 4, 12, 3, 9, 1, 2, 10, 7, 5, 8, 14, 0, 6, 13]


In [ ]:
sentences_1_preproc[0]

['según',
 'el',
 'sondeo',
 'de',
 'los',
 'católicos',
 'cree',
 'que',
 'francisco',
 'está',
 'haciendo',
 'un',
 'buen',
 'excelente',
 'trabajo']

In [148]:
def map_word_embeddings(
        sentence: str,
        sequence_len: int = MAX_LEN,
        fixed_dictionary: Optional[Dictionary] = None
) -> np.ndarray:
    """
    Map to word-embedding indices
    :param sentence:
    :param sequence_len:
    :param fixed_dictionary:
    :return:
    """
    sentence_preproc = simple_preprocess(sentence)[:sequence_len]
    _vectors = np.zeros(sequence_len, dtype=np.int32)
    index = 0
    for word in sentence_preproc:
        if fixed_dictionary is not None:
            if word in fixed_dictionary.token2id:
                # Sumo 1 porque el valor 0 está reservado a padding
                _vectors[index] = fixed_dictionary.token2id[word] + 1
                index += 1
        else:
            if word in wv_model.key_to_index:
                _vectors[index] = wv_model.key_to_index[word] + 1
                index += 1
    return _vectors


def map_pairs(
    sentence_pairs: List[Tuple[str, str, float]],
    sequence_len: int = MAX_LEN,
    fixed_dictionary: Optional[Dictionary] = None
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        vector1 = map_word_embeddings(sentence_1, sequence_len, fixed_dictionary)
        vector2 = map_word_embeddings(sentence_2, sequence_len, fixed_dictionary)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [149]:
mapped_train = map_pairs(input_pairs, fixed_dictionary=diccionario if REMAP_EMBEDDINGS else None)
mapped_val = map_pairs(input_pairs_val, fixed_dictionary=diccionario if REMAP_EMBEDDINGS else None)
mapped_test = map_pairs(input_pairs_test, fixed_dictionary=diccionario if REMAP_EMBEDDINGS else None)

In [150]:
mapped_train[0]

((array([12,  5, 13,  4, 10,  2,  3, 11,  8,  6,  9, 15,  1,  7, 14,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
  array([   5,    4,   10,    2,   21,  958,  381,    5,   14,   21, 1279,
         1158,   62, 1915, 3328,    5, 1014,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,   

In [16]:
# Define model 1
import tensorflow as tf
def model_1(
    input_length: int = MAX_LEN,
    dictionary_size: int = 1000,
    embedding_size: int = 16,
    pretrained_weights: Optional[np.ndarray] = None,
    learning_rate: float = 1e-3,
    trainable: bool = False,
    use_cosine: bool = False,
) -> tf.keras.Model:
    # Input layers
    input_1 = tf.keras.Input(shape=(input_length,), dtype=tf.int32)
    input_2 = tf.keras.Input(shape=(input_length,), dtype=tf.int32)

    # Embedding layer
    if pretrained_weights is None:
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True
        )
    else:
        dictionary_size = pretrained_weights.shape[0]
        embedding_size = pretrained_weights.shape[1]
        initializer = tf.keras.initializers.Constant(pretrained_weights)
        embedding = tf.keras.layers.Embedding(
            dictionary_size,
            embedding_size,
            input_length=input_length,
            mask_zero=True,
            embeddings_initializer=initializer,
            trainable=trainable,
        )

    # Apply embedding to input sequences
    embedded_1 = embedding(input_1)
    embedded_2 = embedding(input_2)

    # Global average pooling
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    pooled_1 = tf.keras.layers.GlobalAveragePooling1D()(embedded_1, mask=_input_mask_1)
    pooled_2 = tf.keras.layers.GlobalAveragePooling1D()(embedded_2, mask=_input_mask_2)

    # Compute similarity/distance
    if use_cosine:   
        # Compute the cosine distance using a Lambda layer
        def cosine_distance(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))
        output = tf.keras.layers.Lambda(cosine_distance)([pooled_1, pooled_2])
    else:
        # Compute the cosine distance using a Lambda layer
        def normalized_product(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return x1_normalized * x2_normalized
    
        output = tf.keras.layers.Lambda(normalized_product)([pooled_1, pooled_2])
        output = tf.keras.layers.Dropout(0.1)(output)
        output = tf.keras.layers.Dense(
            16,
            activation="relu",
        )(output)
        output = tf.keras.layers.Dropout(0.2)(output)
        output = tf.keras.layers.Dense(
            1,
            activation="sigmoid",
        )(output)
        
        output = tf.keras.layers.Lambda(lambda x: x * 5)(output)

    # Define the model
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate))

    return model

In [151]:
class MyLayer_mask(tf.keras.layers.Layer):
    def call(self, x):
        return tf.not_equal(x, 0)
    
class MyLayer_exp(tf.keras.layers.Layer):
    def call(self, x):
        return tf.exp(x)
    
class MyLayer_cast(tf.keras.layers.Layer):
    def call(self, x):
        return tf.cast(x, tf.float32)
    
class MyLayer_reduce_sum(tf.keras.layers.Layer):
    def call(self, x):
        return tf.reduce_sum(x, axis=1, keepdims=True)
    
def model_2(
    input_length: int = MAX_LEN,
    dictionary_size: int = 1000,
    embedding_size: int = 16,
    learning_rate: float = 1e-3,
    pretrained_weights: Optional[np.ndarray] = None,
    trainable: bool = False,
    use_cosine: bool = False,
) -> tf.keras.Model:
    # Inputs
    input_1 = tf.keras.Input((input_length,), dtype=tf.int32)
    input_2 = tf.keras.Input((input_length,), dtype=tf.int32)

    # Embedding Layer
    if pretrained_weights is None:
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True
        )
    else:
        dictionary_size = pretrained_weights.shape[0]
        embedding_size = pretrained_weights.shape[1]
        initializer = tf.keras.initializers.Constant(pretrained_weights)
        embedding = tf.keras.layers.Embedding(
            dictionary_size,
            embedding_size,
            input_length=input_length,
            mask_zero=True,
            embeddings_initializer=initializer,
            trainable=trainable,
        )

    # Embed the inputs
    embedded_1 = embedding(input_1)
    embedded_2 = embedding(input_2)
    # Pass through the embedding layer
    _input_mask_1, _input_mask_2 = MyLayer_mask()(input_1), MyLayer_mask()(input_2)

    # Attention Mechanism
    attention_mlp = tf.keras.Sequential([
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(16, activation='tanh'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    # Apply attention to each embedding
    attention_weights_1 = attention_mlp(embedded_1)  
    attention_weights_2 = attention_mlp(embedded_2) 
    # Mask the attention weights
    attention_weights_1 = MyLayer_exp()(attention_weights_1) * MyLayer_cast()(_input_mask_1[:, :, None])
    attention_weights_2 = MyLayer_exp()(attention_weights_2) * MyLayer_cast()(_input_mask_2[:, :, None])
    # Normalize attention weights
    attention_weights_1 = attention_weights_1 / MyLayer_reduce_sum()(attention_weights_1)
    attention_weights_2 = attention_weights_2 / MyLayer_reduce_sum()(attention_weights_2)
    # Compute context vectors
    projected_1 = MyLayer_reduce_sum()(embedded_1 * attention_weights_1) 
    projected_2 = MyLayer_reduce_sum()(embedded_2 * attention_weights_2) 
    
    if use_cosine:
        # Compute the cosine distance using a Lambda layer
        def cosine_distance(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))
        output = tf.keras.layers.Lambda(cosine_distance)([projected_1, projected_2])
    else:
         # Compute the cosine distance using a Lambda layer
        def normalized_product(x):
            x1, x2 = x
            x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
            x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
            return x1_normalized * x2_normalized
    
        output = tf.keras.layers.Lambda(normalized_product)([projected_1, projected_2])
        output = tf.keras.layers.Dropout(0.1)(output)
        output = tf.keras.layers.Dense(
            16,
            activation="relu",
        )(output)
        output = tf.keras.layers.Dropout(0.2)(output)
        output = tf.keras.layers.Dense(
            1,
            activation="sigmoid",
        )(output)
        
        output = tf.keras.layers.Lambda(lambda x: x * 5)(output)
    # Model Definition
    model = tf.keras.Model(inputs=(input_1, input_2), outputs=output)
    model.compile(
        loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    )
    return model


In [152]:
# Definir constantes de entrenamiento
batch_size: int = 64
num_epochs: int = 128

In [153]:
# Obtener x_train e y_train
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.row_stack(_x_1), np.row_stack(_x_2)), np.array(_y)

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped_train)
x_val, y_val = pair_list_to_x_y(mapped_val)

In [154]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [155]:
_pretrained_weights: Optional[np.ndarray] = None
if USE_PRETRAINED:
    if REMAP_EMBEDDINGS:
        _pretrained_weights = np.zeros(
            (len(diccionario.token2id) + 1, wv_model.vector_size),  dtype=np.float32)
        for token, _id in diccionario.token2id.items():
            if token in wv_model:
             _pretrained_weights[_id + 1] = wv_model[token]

            else:
                # In W2V, OOV will not have a representation. We will use 0.
                pass
    else:
        # Not recommended (this will consume A LOT of RAM)
        _pretrained_weights = np.zeros((wv_model.vectors.shape[0] + 1, wv_model.vector_size,),  dtype=np.float32)
        _pretrained_weights[1:, :] = wv_model.vectors

In [156]:
_pretrained_weights[:5,:]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.04128033, -0.07490284,  0.01133435, ..., -0.02787316,
         0.05126253,  0.03428745],
       [-0.01085784, -0.0409853 ,  0.07754861, ...,  0.10519982,
        -0.01190742, -0.02908356],
       [-0.04239759, -0.10544126, -0.03021094, ..., -0.08874904,
        -0.04355696, -0.00202177],
       [ 0.05474949,  0.01118937,  0.19100004, ...,  0.00659258,
        -0.00208225, -0.02297056]], dtype=float32)

In [25]:
# Build and compile the model
model = model_1(pretrained_weights=_pretrained_weights, trainable=False, use_cosine=False, )
model.summary()

ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [ ]:
# Train the model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Epoch 1/128
21/21 [==============================] - 0s 5ms/step - loss: 1.1811 - val_loss: 1.3921
Epoch 2/128
21/21 [==============================] - 0s 3ms/step - loss: 1.1211 - val_loss: 1.3357
Epoch 3/128
21/21 [==============================] - 0s 2ms/step - loss: 1.0836 - val_loss: 1.2994
Epoch 4/128
21/21 [==============================] - 0s 4ms/step - loss: 1.0656 - val_loss: 1.2791
Epoch 5/128
21/21 [==============================] - 0s 2ms/step - loss: 1.0609 - val_loss: 1.2705
Epoch 6/128
21/21 [==============================] - 0s 2ms/step - loss: 1.0503 - val_loss: 1.2660
Epoch 7/128
21/21 [==============================] - 0s 3ms/step - loss: 1.0624 - val_loss: 1.2646
Epoch 8/128
21/21 [==============================] - 0s 2ms/step - loss: 1.0499 - val_loss: 1.2627
Epoch 9/128
21/21 [==============================] - 0s 2ms/step - loss: 1.0495 - val_loss: 1.2607
Epoch 10/128
21/21 [==============================] - 0s 2ms/step - loss: 1.0541 - val_loss: 1.2592
Epoch 11/

In [ ]:
from scipy.stats import pearsonr

x_test, y_test = pair_list_to_x_y(mapped_test)
def compute_pearson(x_, y_, model):
    # Get predictions for the model
    y_pred = model.predict(x_)
    # Compute pearson correlation
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation

In [ ]:
# Print results
print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train, model)}")
print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val, model)}")
print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test, model)}")


42/42 [==============================] - 0s 2ms/step
Correlación de Pearson (train): 0.514176777517245
3/3 [==============================] - 0s 2ms/step
Correlación de Pearson (validation): 0.6268977035073102
5/5 [==============================] - 0s 2ms/step
Correlación de Pearson (test): 0.46831927978262927


Alternative model

In [157]:
# Build Model 2
model2 = model_2(pretrained_weights=_pretrained_weights, trainable=False, use_cosine=False)
model2.summary()

c:\Users\ralva\Desktop\University\4t-Cuatrimestre-GIA\PLH\PLH-WordEmbeddings\WordEmbeddings\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
c:\Users\ralva\Desktop\University\4t-Cuatrimestre-GIA\PLH\PLH-WordEmbeddings\WordEmbeddings\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'sequential_29' (of type Sequential) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional_39"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_140     │ (None, 96)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_141     │ (None, 96)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_55        │ (None, 96, 300)   │  3,437,100 │ input_layer_140[… │
│ (Embedding)         │                   │            │ input_layer_141[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_108       │ (None, 96)        │          0 │ input_layer_140[… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_mask_23    │ (None, 96)        │          0 │ input_layer_140[… │
│ (MyLayer_mask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_109       │ (None, 96)        │          0 │ input_layer_141[… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_mask_24    │ (None, 96)        │          0 │ input_layer_141[… │
│ (MyLayer_mask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_29       │ (None, 96, 1)     │      4,833 │ embedding_55[0][… │
│ (Sequential)        │                   │            │ not_equal_108[0]… │
│                     │                   │            │ embedding_55[1][… │
│                     │                   │            │ not_equal_109[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_37         │ (None, 96, 1)     │          0 │ my_layer_mask_23… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_38         │ (None, 96, 1)     │          0 │ my_layer_mask_24… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_exp_14     │ (None, 96, 1)     │          0 │ sequential_29[0]… │
│ (MyLayer_exp)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_cast_14    │ (None, 96, 1)     │          0 │ get_item_37[0][0] │
│ (MyLayer_cast)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_exp_15     │ (None, 96, 1)     │          0 │ sequential_29[1]… │
│ (MyLayer_exp)       │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ my_layer_cast_15    │ (None, 96, 1)     │          0 │ get_item_38[0][0] │
│ (MyLayer_cast)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_21         │ (None, 96, 1)     │          0 │ my_layer_exp_14[… │
│ (Multiply)          │                   │            │ my_layer_cast_14… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_22         │ (None, 96, 1)     │          0 │ my_layer_exp_15[… │
│ (Multiply)          │                   │            │ my_layer_cast_15

 Total params: 3,446,766 (13.15 MB)

 Trainable params: 9,666 (37.76 KB)

 Non-trainable params: 3,437,100 (13.11 MB)

In [158]:
# Train model 2
model2.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

Epoch 1/128


c:\Users\ralva\Desktop\University\4t-Cuatrimestre-GIA\PLH\PLH-WordEmbeddings\WordEmbeddings\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'my_layer_exp_15' (of type MyLayer_exp) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
c:\Users\ralva\Desktop\University\4t-Cuatrimestre-GIA\PLH\PLH-WordEmbeddings\WordEmbeddings\Lib\site-packages\keras\src\layers\layer.py:877: UserWarning: Layer 'my_layer_exp_14' (of type MyLayer_exp) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


21/21 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 3.9506 - val_loss: 2.4358
Epoch 2/128
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2.1612 - val_loss: 1.7771
Epoch 3/128
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1.3721 - val_loss: 1.4499
Epoch 4/128
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1.2205 - val_loss: 1.3820
Epoch 5/128
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 1.1595 - val_loss: 1.3612
Epoch 6/128
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1.1187 - val_loss: 1.3506
Epoch 7/128
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1.0945 - val_loss: 1.3347
Epoch 8/128
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1.1008 - val_loss: 1.3131
Epoch 9/128
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 1.0910 - val_loss: 1.3133
Epoch 10/128
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1.0820 - val_loss: 1.3068
Epoch 11/128
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 1.0724 - val_loss: 1.3091
Epoch 12/128
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1.0

In [159]:
# Print results
print(f"Correlación de Pearson (train): {compute_pearson(x_train, y_train, model2)}")
print(f"Correlación de Pearson (validation): {compute_pearson(x_val, y_val, model2)}")
print(f"Correlación de Pearson (test): {compute_pearson(x_test, y_test, model2)}")

42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Correlación de Pearson (train): 0.2117090201762222
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Correlación de Pearson (validation): -0.1544596696880599
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Correlación de Pearson (test): 0.11921612321701991
